In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
import os
from PIL import Image
from torch.utils.data import Dataset

In [2]:
# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((64, 64)),  # Adjust based on your image size
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Custom dataset class to handle image loading and labeling
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [4]:
# Helper function to load images and assign labels (edible = 0, poisonous = 1)
def load_images_from_folder(folder_path, label, transform):
    image_paths = []
    labels = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust extensions if needed
            img_path = os.path.join(folder_path, filename)
            image_paths.append(img_path)
            labels.append(label)
    
    return CustomImageDataset(image_paths, labels, transform)


In [5]:
# Load edible and poisonous datasets
train_edible_dataset = load_images_from_folder('CNN_Dataset/edible/train', label=0, transform=data_transforms['train'])
test_edible_dataset = load_images_from_folder('CNN_Dataset/edible/test', label=0, transform=data_transforms['test'])
train_poisonous_dataset = load_images_from_folder('CNN_Dataset/poisonous/train', label=1, transform=data_transforms['train'])
test_poisonous_dataset = load_images_from_folder('CNN_Dataset/poisonous/test', label=1, transform=data_transforms['test'])


In [6]:
# Combine edible and poisonous datasets into single training and testing datasets
train_dataset = ConcatDataset([train_edible_dataset, train_poisonous_dataset])
test_dataset = ConcatDataset([test_edible_dataset, test_poisonous_dataset])

In [7]:
# Data loaders for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
# Number of classes (edible = 0, poisonous = 1)
num_classes = 2
model = CNNModel(num_classes=num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Softmax function to compute confidence percentages
softmax = nn.Softmax(dim=1)

In [10]:
# Training loop
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

        # Test step
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_acc = correct / total
        print(f'Test Accuracy: {test_acc * 100}%')

    print('Training finished')

In [11]:
# Prediction function with confidence percentage
def predict_with_confidence(model, image_tensor):
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor.unsqueeze(0))  # Add batch dimension
        probabilities = softmax(outputs)
        confidence, predicted_class = torch.max(probabilities, 1)
        return predicted_class.item(), confidence.item()

In [12]:
# Train the model
train_model(
    model,
    train_loader,
    test_loader,
    criterion,
    optimizer,
    num_epochs = 20
)

Epoch 1, Loss: 0.756011093656222
Test Accuracy: 55.09138381201044%
Epoch 2, Loss: 0.6906295704344908
Test Accuracy: 54.83028720626631%
Epoch 3, Loss: 0.6864449692269167
Test Accuracy: 57.702349869451695%
Epoch 4, Loss: 0.6719959365824858
Test Accuracy: 56.13577023498695%
Epoch 5, Loss: 0.6628067505856355
Test Accuracy: 53.78590078328982%
Epoch 6, Loss: 0.6448631944755713
Test Accuracy: 49.869451697127936%
Epoch 7, Loss: 0.6288647713760535
Test Accuracy: 50.65274151436031%
Epoch 8, Loss: 0.6073748047153155
Test Accuracy: 55.61357702349869%
Epoch 9, Loss: 0.5703236808379492
Test Accuracy: 51.697127937336816%
Epoch 10, Loss: 0.5240388655414184
Test Accuracy: 56.6579634464752%
Epoch 11, Loss: 0.4617565864076217
Test Accuracy: 53.26370757180157%
Epoch 12, Loss: 0.3993265153840184
Test Accuracy: 56.13577023498695%
Epoch 13, Loss: 0.3286446997274955
Test Accuracy: 56.6579634464752%
Epoch 14, Loss: 0.27231980363527936
Test Accuracy: 53.78590078328982%
Epoch 15, Loss: 0.22016861910621324
Test A

In [13]:
torch.save(model.state_dict(), 'CNN_torch.pth')  # Save the model

### Testing the model

In [14]:
# Load and preprocess the image
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')  # Ensure RGB format
    image = transform(image)  # Apply transformations
    return image

In [15]:
# Example of predicting a single image
image_path = 'CNN_Dataset/Poisonous/test/1667816120_xylaria-carpophila-4.jpg'  # Replace with your image path
image_tensor = preprocess_image(image_path, data_transforms['test'])  # Use the test transforms

# Predict the class and confidence
predicted_class, confidence = predict_with_confidence(model, image_tensor)

# Map the predicted class to label
class_mapping = {0: 'edible', 1: 'poisonous'}
print(f'Predicted Class: {class_mapping[predicted_class]}, Confidence: {confidence * 100:.2f}%')

Predicted Class: poisonous, Confidence: 51.50%


In [16]:
# Example of predicting a single image
image_path = 'CNN_Dataset/Edible/test/psilocybe-samuiensis-1.jpg'  # Replace with your image path
image_tensor = preprocess_image(image_path, data_transforms['test'])  # Use the test transforms

# Predict the class and confidence
predicted_class, confidence = predict_with_confidence(model, image_tensor)

# Map the predicted class to label
class_mapping = {0: 'edible', 1: 'poisonous'}
print(f'Predicted Class: {class_mapping[predicted_class]}, Confidence: {confidence * 100:.2f}%')

Predicted Class: edible, Confidence: 99.95%


In [17]:
# Example of predicting a single image
image_path = 'mushroom_images/Test/Poisonous.jpeg'  # Replace with your image path
image_tensor = preprocess_image(image_path, data_transforms['test'])  # Use the test transforms

# Predict the class and confidence
predicted_class, confidence = predict_with_confidence(model, image_tensor)

# Map the predicted class to label
class_mapping = {0: 'edible', 1: 'poisonous'}
print(f'Predicted Class: {class_mapping[predicted_class]}, Confidence: {confidence * 100:.2f}%')

Predicted Class: poisonous, Confidence: 99.58%


In [18]:
# Example of predicting a single image
image_path = 'mushroom_images/Test/Edible.jpeg'  # Replace with your image path
image_tensor = preprocess_image(image_path, data_transforms['test'])  # Use the test transforms

# Predict the class and confidence
predicted_class, confidence = predict_with_confidence(model, image_tensor)

# Map the predicted class to label
class_mapping = {0: 'edible', 1: 'poisonous'}
print(f'Predicted Class: {class_mapping[predicted_class]}, Confidence: {confidence * 100:.2f}%')

Predicted Class: edible, Confidence: 100.00%


In [19]:
# Example of predicting a single image
image_path = 'CNN_Dataset/Edible/test/gymnopilus-luteofolius-4.jpg'  # Replace with your image path
image_tensor = preprocess_image(image_path, data_transforms['test'])  # Use the test transforms

# Predict the class and confidence
predicted_class, confidence = predict_with_confidence(model, image_tensor)

# Map the predicted class to label
class_mapping = {0: 'edible', 1: 'poisonous'}
print(f'Predicted Class: {class_mapping[predicted_class]}, Confidence: {confidence * 100:.2f}%')

Predicted Class: poisonous, Confidence: 95.32%


In [20]:
# Example of predicting a single image
image_path = 'CNN_Dataset/Edible/test/gymnopilus-luteofolius-4.jpg'  # Replace with your image path
image_tensor = preprocess_image(image_path, data_transforms['test'])  # Use the test transforms

# Predict the class and confidence
predicted_class, confidence = predict_with_confidence(model, image_tensor)

# Map the predicted class to label
class_mapping = {0: 'edible', 1: 'poisonous'}
print(f'Predicted Class: {class_mapping[predicted_class]}, Confidence: {confidence * 100:.2f}%')

Predicted Class: poisonous, Confidence: 95.32%
